In [ ]:
#@title # Mount Google Drive { vertical-output: true }

%cd /content/
# fixes weird permissions error with gdown
!pip install --upgrade --no-cache-dir gdown

from google.colab import drive
import time

drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#@title # Install tinycudann, nerfstudio, deps { vertical-output: true }

%cd /content/
!pip install --upgrade pip

# Building takes too long, so we these grab pre-built from wheelhouse
!pip install --no-index --find-links=/content/gdrive/MyDrive/colab/wheelhouse tinycudann nerfstudio

# Install localtunnel
# We are using localtunnel https://github.com/localtunnel/localtunnel but ngrok could also be used
!npm install -g localtunnel

In [ ]:
#@title # Config { vertical-output: true }

scene = "lego"
dataset = "dnerf"
gd_scene_dir = f"/content/gdrive/MyDrive/colab/data/{dataset}/{scene}"
scene_dir          = f"/content/data/{dataset}/{scene}"
contains_scene_dir = f"/content/data/{dataset}"
experiment_name = "experiment_01"


In [20]:
#@title # Copy dataset { vertical-output: true }

!mkdir -p $contains_scene_dir
!cp -r $gd_scene_dir $contains_scene_dir

In [ ]:
#@title # Set up and Start Viewer { vertical-output: true }

%cd /content

# Tunnel port 7007, the default for
!rm url.txt 2> /dev/null
get_ipython().system_raw('lt --port 7007 >> url.txt 2>&1 &')

import time
time.sleep(3) # the previous command needs time to write to url.txt

with open('url.txt') as f:
  lines = f.readlines()
websocket_url = lines[0].split(": ")[1].strip().replace("https", "wss")
url = f"https://viewer.nerf.studio/?websocket_url={websocket_url}"
print(url)

In [ ]:
#@title # Start Training { vertical-output: true }

import os
from IPython.core.display import display, HTML

%cd /content

!ns-train \
  nerfplayer-nerfacto \
    --experiment-name $experiment_name \
    --viewer.websocket-port 7007 \
    --pipeline.model.proposal-initial-sampler uniform \
    --pipeline.model.near-plane 2. \
    --pipeline.model.far-plane 6. \
    --pipeline.datamanager.camera-optimizer.mode off \
    --pipeline.model.use-average-appearance-embedding False \
    --pipeline.model.distortion-loss-mult 0 \
    --pipeline.model.disable-scene-contraction True \
  dnerf-data \
    --data $scene_dir \
> stdout.txt 2> stderr.txt

/content


In [ ]:
#@title # Render Video { vertical-output: true }
#@markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
#@markdown <h5>The rendered video should be at renders/output.mp4!</h5>

from google.colab import files
import os
from IPython.core.display import display, HTML

render_dir = "/content/gdrive/MyDrive/colab/renders"
!mkdir $render_dir
render_path = f"{render_dir}/output.mp4"

base_dir = f"/content/outputs/{experiment_name}/nerfacto" 
training_run_dir = base_dir + "/" + os.listdir(base_dir)[0]

display(HTML('<h3>Upload the camera path JSON.</h3>'))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render \
  --load-config $config_filename \
  --traj filename \
  --camera-path-filename $camera_path_filename \
  --output-path $render_path #> stdout.txt